In [ ]:
from google.colab import drive
drive.mount('/content/drive')	
import os
os.chdir('/content/drive/MyDrive/gpl new')
datapath='./data/'

In [ ]:
%pip install deap
%pip install scoop
#%% import packages
import numpy as np
import pandas as pd
from statsmodels import regression
import statsmodels.api as sm
import operator
import math
import random
from datetime import datetime
import os
from tqdm import tqdm
import deap
from deap import base, creator, gp, tools, algorithms
import pickle
import matplotlib.pyplot as plt
from scoop import futures
plt.rcParams["font.sans-serif"] = ["SimHei"]

In [ ]:
#%% functions
def read_pickle(str1):
    with open(datapath + "{}.pkl".format(str1), "rb") as f:
        data = pickle.load(f)
    return data


def store_pickle(data, str1):
    with open(datapath + "{}.pkl".format(str1), "wb") as f:
        pickle.dump(data, f, protocol=4)


def train_test_split(data, train_ratio):
    return (
        data.iloc[: round(train_ratio * (len(data)))],
        data.iloc[round(train_ratio * (len(data))) :],
    )




In [ ]:
features = ['NEWS_SENTIMENT_DAILY_AVG',
 'PE_RATIO',
 'IS_DILUTED_EPS',
 'CASH_RATIO',
 'EBIT',
 'ASSET_TURNOVER',
 'PX_HIGH',
 'PX_LOW',
 'PX_OPEN',
 'BLOOMBERG_CLOSE_PRICE',
 'PX_VOLUME',
 'CUR_MKT_CAP']

In [ ]:
data = []
close = read_pickle('close_price')
for feature in features:
  data.append(read_pickle(feature))
dat = []
dat_test = []
for d in data:
  tmp,tmp_test = train_test_split(d,0.7)
  dat.append(tmp)
  dat_test.append(tmp_test)

close_train,close_test=train_test_split(close,0.7)
rt_20_train = close_train.pct_change(20).ffill()
rt_20_test = close_test.pct_change(20).ffill()

In [ ]:
close.describe()

In [ ]:
#%% layer operate 
import OperatorFunc

Layer1 = gp.PrimitiveSet("Layer1", 12)
Layer1.addPrimitive(OperatorFunc.Add, 2)  # the number of operator
Layer1.addPrimitive(OperatorFunc.Sub, 2)
Layer1.addPrimitive(OperatorFunc.Mul, 2)
Layer1.addPrimitive(OperatorFunc.Div, 2)
Layer1.addPrimitive(OperatorFunc.min_, 2)
Layer1.addPrimitive(OperatorFunc.max_, 2)
Layer1.addPrimitive(OperatorFunc.inv, 1)
Layer1.addPrimitive(OperatorFunc.sqrt, 1)
Layer1.addPrimitive(OperatorFunc.sin, 1)
Layer1.addPrimitive(OperatorFunc.cos, 1)
Layer1.addPrimitive(OperatorFunc.abslog, 1)
Layer1.addPrimitive(OperatorFunc.mean5, 1)
Layer1.addPrimitive(OperatorFunc.mean10, 1)
Layer1.addPrimitive(OperatorFunc.mean20, 1)
Layer1.addPrimitive(OperatorFunc.std10, 1)
Layer1.addPrimitive(OperatorFunc.std20, 1)
Layer1.addPrimitive(OperatorFunc.max10, 1)
Layer1.addPrimitive(OperatorFunc.max20, 1)
Layer1.addPrimitive(OperatorFunc.min10, 1)
Layer1.addPrimitive(OperatorFunc.min20, 1)
Layer1.addPrimitive(OperatorFunc.shift10, 1)
Layer1.addPrimitive(OperatorFunc.shift20, 1)
Layer1.addPrimitive(OperatorFunc.square, 1)
Layer1.addPrimitive(OperatorFunc.rank10, 1)
Layer1.addPrimitive(OperatorFunc.quantile_3_60, 1)
Layer1.addPrimitive(OperatorFunc.quantile_1_60, 1)
 


Layer1.renameArguments(
    ARG0=features[0], ARG1=features[1], ARG2=features[2], ARG3=features[3], ARG4=features[4], ARG5=features[5],
    ARG6=features[6], ARG7=features[7], ARG8=features[8], ARG9=features[9], ARG10=features[10], ARG11=features[11],
)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

L1box = base.Toolbox()
L1box.register("attr_float", random.random)
L1box.register("expr", gp.genHalfAndHalf, pset=Layer1, min_=1, max_=4)  # can also choose gp.Full here
L1box.register("individual", tools.initIterate, creator.Individual, L1box.expr)
L1box.register("population", tools.initRepeat, list, L1box.individual)
L1box.register("compile", gp.compile, pset=Layer1)
L1box.register("max",futures.map)
L1pop = L1box.population(n=500)

alterbox = base.Toolbox()
alterbox.register("expr", gp.genHalfAndHalf, pset=Layer1, min_=1, max_=5)
alterbox.register("individual", tools.initIterate, creator.Individual, alterbox.expr)


In [ ]:
#%% 目标函数
def fitnessIC(individual, dat, y):
    func = L1box.compile(expr=individual)

    """It can be understand as dataframe calculation which has columns--stocks, index--timestamp"""

    try:
        y_pred = func(dat[0], dat[1], dat[2], dat[3], dat[4], dat[5],dat[6], dat[7], dat[8], dat[9], dat[10], dat[11])
        factor = y_pred.copy()
    except:
        return 0,

    factor.replace([np.inf, -np.inf], np.nan, inplace=True)

    ret = y.copy()
    # mask = y[-1].shift(-1).fillna(True)  | factor.isnull() | y[0].isnull()  
    # factor[mask] = np.nan
    # ret[mask] = np.nan
    #
    # y_pred, ret = y_pred.dropna(axis  = 0, how = 'all').align(ret.dropna(axis  = 0, how = 'all'), join='outer', axis=1)
    # y_pred, ret = y_pred.align(ret, join='inner', axis=0)
    IC_series = factor.corrwith(ret, axis=1).dropna()
    Rank_IC = IC_series.mean() if IC_series.mean() is not np.nan else 0
    return (Rank_IC,)


#%% train the model
print("Start training...")
L1box.register("evaluate", fitnessIC, dat=dat, y=rt_20_train.shift(-21))  #training set is on the left
L1box.register("select", tools.selTournament, tournsize=3)


L1box.register("mate", gp.cxOnePoint)
L1box.register("expr_mut", gp.genFull, min_=0, max_=2)
L1box.register("mutate", gp.mutUniform, expr=L1box.expr_mut, pset=Layer1)

L1box.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
L1box.decorate(
    "mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17)
)

n_population = 100
n_generation = 30

pop = L1box.population(n=n_population)
hof = tools.HallOfFame(n_population * n_generation)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

In [ ]:
pop, log = algorithms.eaSimple(
    pop,
    L1box,
    cxpb=0.5,
    mutpb=0.2,
    ngen=n_generation,
    stats=stats,
    halloffame=hof,
    verbose=True,
)
#%% best
print(str(hof.items[0]))
print(hof.keys[0])
print(hof.items[0].fitness)


In [ ]:
hof = read_pickle('hof')

In [ ]:
for i in hof.items:
  print(str(i))

In [ ]:
from tqdm import tqdm
data = pd.DataFrame()
func = L1box.compile(expr=hof.items[0])
y_pred = func(dat[0], dat[1], dat[2], dat[3], dat[4], dat[5],dat[6], dat[7], dat[8], dat[9], dat[10], dat[11])
ind = y_pred.stack().index

for digit in tqdm(range(0,300)):
  func = L1box.compile(expr=hof.items[digit])
  y_pred = func(dat[0], dat[1], dat[2], dat[3], dat[4], dat[5],dat[6], dat[7], dat[8], dat[9], dat[10], dat[11])
  data[digit]=y_pred.stack().reindex(ind).values

In [ ]:
co=data.corr()

In [ ]:
co.to_csv(r'corr.csv')

In [ ]:


#%% test function
def cal_ic(rt_dic, plotting=True):
    print("starting factor analysis...")

    ic_list = []
    for period in {1, 5, 10, 20}:
        ic_list.append(
            factor_df.apply(lambda x: x.corr(rt_dic[period].loc[x.name]), axis=1)
        )

    ic_df = pd.concat(ic_list, axis=1).dropna(how="all")
    ic_df.columns = [1, 5, 10, 20]
    ic_summary_table = pd.concat(
        [ic_df.mean(), ic_df.std(), np.sqrt(len(ic_df)) * (ic_df.mean() / ic_df.std())],
        axis=1,
        keys=["IC mean", "IC std", "IC T-Statistics"],
    ).T
    print(ic_summary_table)

    if plotting:
        ic_df.rolling(21).mean().shift(-11).plot(
            title="monthly mean IC",
            kind="area",
            stacked=False,
            figsize=(20, 10),
            colormap="plasma",
            grid=True,
        )
        plt.show()
    return ic_df

def cal_group_ret(rt_dic,y_pred, n=10):
    print("factor grouping analysis")
    group = y_pred.apply(lambda x:pd.cut(x,bins = n, labels = range(1,n+1)))
    mean_ret_dict = {}
    for period in rt_dic.keys():
        mean_ret_dict[period] = group.apply(
            lambda x: rt_dic[period].loc[x.name].groupby(x).mean() / period, axis=1,
        )
    mean_ret_df = pd.concat(
        [21 * (mean_ret_dict[p].mean()) for p in rt_dic.keys()],
        axis=1,
        keys=rt_dic.keys(),
    )
    mean_ret_df.plot(kind="bar", figsize=(20, 8), title="monthly mean return per group")
    plt.show()

    long_short_nvs = pd.concat(
        [
            (
                mean_ret_dict[p].iloc[:, n - 1] - mean_ret_dict[p].iloc[:, 0] + 1
            ).cumprod()
            for p in rt_dic.keys()
        ],
        axis=1,
        keys=rt_dic.keys(),
    )
    long_short_nvs.plot(
        title="long short net value", figsize=(20, 10), colormap="plasma", grid=True
    )
    plt.show()
    return mean_ret_dict



In [ ]:
digit=0
func = L1box.compile(expr=hof.items[digit])

y_pred = func(dat[0], dat[1], dat[2], dat[3], dat[4], dat[5],dat[6], dat[7], dat[8], dat[9], dat[10], dat[11])
#%% insample test
factor_df = y_pred.copy()
rt_dic = {}
for i in {1, 5, 10, 20}:
    rt_dic[i] = close_train.ffill().pct_change(i).shift(-i - 1)

ic_df = cal_ic(rt_dic)
cal_group_ret(rt_dic,y_pred)
#%% outsample test
y_test = func(
    dat_test[0], dat_test[1], dat_test[2], dat_test[3], dat_test[4], dat_test[5],
    dat_test[6], dat_test[7], dat_test[8], dat_test[9], dat_test[10], dat_test[11],
)
factor_df = y_test.copy()
rt_dic = {}
for i in {1, 5, 10, 20}:
    rt_dic[i] = close_test.ffill().pct_change(i).shift(-i - 1)
ic_df_test = cal_ic(rt_dic)
cal_group_ret(rt_dic,y_test)

In [ ]:
store_pickle(hof,'hof')